In [1]:
import os


In [2]:
pwd

'd:\\PW_DS\\Deep Learning\\End-To-End-Chicken-Disease-Classification-Project-using-CNN\\research'

In [3]:
os.chdir('../')

In [4]:
pwd

'd:\\PW_DS\\Deep Learning\\End-To-End-Chicken-Disease-Classification-Project-using-CNN'

## Entity

In [5]:
from dataclasses import  dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epoch: int
    params_batch_size: int
    params_is_augmented: bool
    params_image_size: list
    data_dir: Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


## Config Manager

In [7]:
from chicken_disease_classification.constants import  *
from chicken_disease_classification.utils.common import  read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:

        config = self.config.prepare_callbacks

        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = self.config.artifacts_root,
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )


        return prepare_callbacks_config







    def get_training_data_config(self) -> TrainingConfig:
        training = self.config.training
        
        prepare_base_model = self.config.prepare_base_model

        params = self.params.vgg_16

        training_data = training.data_dir


        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epoch = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmented = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE,
            data_dir = Path(training.data_dir)
        )

        return training_config


## Components

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [10]:
class PrepareCallbacks:
    def __init__(
        self,
        config: PrepareCallbacksConfig):

        self.config = config
    

    @property
    def create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_runnng_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}'
        )

        return tf.keras.callbacks.TensorBoard(log_dir = tb_runnng_log_dir)

    @property
    def create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = str(self.config.checkpoint_model_filepath),
            save_best_only = True,
            # save_weights_only = False
        )


    @property
    def get_tb_ckpt_callbacks(self):
        return [self.create_tb_callbacks,self.create_ckpt_callbacks]
        

    


In [18]:
class Training:
    def __init__(
        self,
        config: TrainingConfig):

        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )


    def train_valid_generator(self):

        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        # test_data_generator_kwargs = dict(
        #     rescale = 1./255,
        #     validation_split = 0.20
        # )



        data_flow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            # shuffle = True,
            interpolation = 'bilinear'
        )

        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )


        self.valid_generator = valid_data_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            class_mode='categorical',
            **data_flow_kwargs
        )


        # test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        #     **test_data_generator_kwargs
        # )


        # self.test_generator = test_data_generator.flow_from_directory(
        #     data = self.valid_generator,
        #     subset = 'validation',
        #     shuffle = False,
        #     class_mode='categorical',
        #     **data_flow_kwargs
        # )



        if self.config.params_is_augmented:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **data_generator_kwargs
            )

        else:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                **data_generator_kwargs
            )
        
        self.train_generator = train_data_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'training',
            shuffle = True,
            class_mode='categorical',
            **data_flow_kwargs
        )

    

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)
    

    def train(self,callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size

        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size 


        self.model.fit(
            self.train_generator,
            steps_per_epoch = self.steps_per_epoch,
            epochs = self.config.params_epoch,
            validation_data = self.valid_generator,
            validation_steps = self.validation_steps,
            callbacks = callback_list
        )


        self.save_model(
            path = self.config.trained_model_path,
            model =self.model
        )




## Pipeline

In [20]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callbacks_list = prepare_callbacks.get_tb_ckpt_callbacks

    

    training_config = config.get_training_data_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    # training.train(callback_list=callbacks_list)

except Exception as e:
    raise e


    

[2024-01-16 20:57:52,010: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-01-16 20:57:52,015: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-01-16 20:57:52,017: INFO: common: Created Directory at artifacts.]
[2024-01-16 20:57:52,019: INFO: common: Created Directory at artifacts\prepare_callbacks\checkpoint_dir.]
[2024-01-16 20:57:52,020: INFO: common: Created Directory at artifacts\prepare_callbacks\tensorboard_log_dir.]
[2024-01-16 20:57:52,022: INFO: common: Created Directory at artifacts\training.]


Found 1230 images belonging to 3 classes.
Found 4923 images belonging to 3 classes.
307/307 [==============================] - 968s 3s/step - loss: 2.1842 - accuracy: 0.8235 - val_loss: 0.8388 - val_accuracy: 0.9046


c:\Users\Maaz\anaconda3\envs\chicken\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
# To see tensorborad logs tyep command in cmd/anaconda prompt

# tensorboard --logdir path/to/tensorboard/log_dir/